In [1]:
!pip install --upgrade psann

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.8/73.8 kB 3.5 MB/s eta 0:00:00


In [2]:
!pip install --upgrade sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 71.3 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3


The Idea is to pull the top 10 stocks from a sector spdr like XLE and trading those, reweighted daily. Also, with the backtest, simply iterate through days.

In [3]:
import pandas as pd
import numpy as np
from IPython.display import clear_output
import yfinance as yf
import matplotlib.pyplot as plt
import itertools
import time
from datetime import datetime, timedelta
import yfinance as yf
import torch
import torch.nn as nn
from psann import ResConvPSANNRegressor, ResPSANNRegressor, PSANNRegressor, PredictiveExtrasConfig, portfolio_log_return_reward, make_predictive_extras_trainer_from_estimator
from psann.augmented import PredictiveExtrasTrainer
from psann.metrics import equity_curve, portfolio_metrics

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
def prices_context_extractor(X_ep: torch.Tensor) -> torch.Tensor:
    return X_ep

def series_reward(tr, X_prices):
    alloc, _ = tr.infer_series(X_prices)
    alloc_b = torch.from_numpy(alloc).unsqueeze(0).float()
    prices_b = torch.from_numpy(X_prices).unsqueeze(0).float()
    with torch.no_grad():
        r = portfolio_log_return_reward(alloc_b, prices_b, trans_cost=cfg.trans_cost)
    return float(r.item())

In [6]:
def yfin(ticker):
  try:
    start_date = datetime.today() - timedelta(days=365*20)
    end_date = datetime.today() #- timedelta(days=365*4)
    tickerData = yf.Ticker(ticker)
    df = tickerData.history(start=start_date, end=end_date).reset_index()
    df = df[['Date','Open','High','Low','Close']]
    df['Ticker'] = ticker

    return df
  except Exception:
    return pd.DataFrame()

In [7]:
df = pd.DataFrame()
tickers = ['XLE', 'XOM', 'CVX', 'COP', 'EOG', 'WMB', 'SLB', 'MPC', 'PSX', 'KMI', 'OKE']

for ticker in tickers:
  sub_df = yfin(ticker)
  sub_df.rename(columns = {'Close':f'{ticker}_Close'}, inplace=True)

  if tickers.index(ticker) == 0:
    df = sub_df[['Date', f'{ticker}_Close']]
  else:
    df = df.merge(sub_df[['Date', f'{ticker}_Close']], on='Date')
  clear_output(wait=True)
  print(tickers.index(ticker))

10


In [8]:
df['USD_Close'] = 1.0
df['Date'] = pd.to_datetime(df['Date'])

In [9]:
price_cols = [c for c in df.columns if c != "Date"]
df = df.sort_values("Date").replace([np.inf, -np.inf], np.nan)
df[price_cols] = df[price_cols].ffill()
df = df.dropna(subset=price_cols)
df[price_cols] = df[price_cols].astype(np.float32)

df_final = df.copy()
split = int(0.9 * len(df_final))
train = df_final.iloc[:split]
test = df_final.iloc[split:]


In [10]:
feature_cols = price_cols
train_prices = train[feature_cols].to_numpy(dtype=np.float32)
test_prices  = test[feature_cols].to_numpy(dtype=np.float32)


We need to add 2-D support with Hisso training.

In [11]:
def sliding_windows(X_flat: np.ndarray, window: int) -> np.ndarray:
    N, F = X_flat.shape
    idx = np.arange(window)[None, :] + np.arange(N - window + 1)[:, None]
    return X_flat[idx]  # (N-window+1, window, F)

In [12]:
T = 20

train_windows = sliding_windows(train_prices, T)
test_windows = sliding_windows(test_prices, T)

train_windows = train_windows[:, None, :, :].astype(np.float32)
test_windows = test_windows[:, None, :, :].astype(np.float32)


In [14]:
C, H, W = train_windows.shape[1:]
FLAT_DIM = C * H * W

def hisso_episode_context(X_episode: torch.Tensor) -> torch.Tensor:
    # HISSO sees flattened windows; ensure strictly positive prices for log reward
    return torch.clamp(X_episode[..., :FLAT_DIM], min=1e-6)

def conv2d_reward(allocations: torch.Tensor, ctx: torch.Tensor) -> torch.Tensor:
    # Map flattened outputs back to (C,H,W) and use the most recent row for returns
    B, T, _ = allocations.shape
    alloc_maps = allocations.reshape(B, T, C, H, W)
    price_maps = ctx.reshape(B, T, C, H, W)
    alloc_last = alloc_maps[:, :, 0, -1, :]
    price_last = price_maps[:, :, 0, -1, :]
    return portfolio_log_return_reward(alloc_last, price_last)


In [15]:
psann = ResConvPSANNRegressor(
    hidden_layers=8,
    conv_channels=16,
    hidden_width=64,
    epochs=6,
    batch_size=64,
    lr=3e-4,
    w0_first=6.0,
    w0_hidden=1.0,
    drop_path_max=0.05,
    scaler="standard",
    extras=0,
    preserve_shape=True,
)
psann.fit(
    train_windows,
    y=None,
    hisso=True,
    hisso_window=32,
    noisy=0.01,
    verbose=1,
    hisso_reward_fn=conv2d_reward,
    hisso_context_extractor=hisso_episode_context,
)


AssertionError: allocations and prices must align

In [ ]:
# Backtest the HISSO-trained strategy against buy-and-hold XLE
test_flat = test_windows.reshape(test_windows.shape[0], -1)
alloc_series, extras_series = psann.hisso_infer_series(test_flat)
alloc_maps = alloc_series.reshape(-1, C, H, W)
alloc_latest = alloc_maps[:, 0, -1, :]
alloc_latest = alloc_latest / np.clip(alloc_latest.sum(axis=1, keepdims=True), 1e-6, None)
price_latest = np.clip(test_windows[:, 0, -1, :], 1e-6, None)
strategy_curve = equity_curve(alloc_latest, price_latest)
xle_idx = price_cols.index('XLE_Close')
buy_hold_curve = price_latest[:, xle_idx] / price_latest[0, xle_idx]
plt.figure(figsize=(10, 5))
plt.plot(strategy_curve, label='PSANN strategy')
plt.plot(buy_hold_curve, label='Buy & Hold XLE')
plt.title('Out-of-sample equity curves')
plt.legend()
plt.grid(True, alpha=0.3)


In [ ]:
# Step-by-step inspection across the test set
dates = test.iloc[T-1:]['Date'].reset_index(drop=True)
lookahead = min(10, len(strategy_curve) - 1)
for i in range(lookahead):
    alloc_vec = alloc_latest[i]
    alloc_vec = alloc_vec / np.maximum(alloc_vec.sum(), 1e-6)
    day_prices = price_latest[i]
    next_prices = price_latest[i + 1]
    asset_ret = next_prices / day_prices - 1.0
    port_ret = float((alloc_vec * asset_ret).sum())
    print(f"{dates.iloc[i].date()}: return={port_ret:.4f}, top weights")
    top_idx = np.argsort(-alloc_vec)[:3]
    for j in top_idx:
        print(f"    {price_cols[j]} weight={alloc_vec[j]:.3f} ret={asset_ret[j]:.4f}")
    print('-' * 40)
